In [28]:
cd /home/jovyan/capstone/GetOldTweets-python/

/home/jovyan/capstone/GetOldTweets-python


In [29]:
import pandas as pd
import got3 as got
from pprint import pprint 

### Looking at JSON file to see if I can filter out tweets that are replies to other tweets. Also looking for geo and username fields.

**Update:**  Added functionality to skip replies. Found username field but couldn't find any location information (geo, coordinates, location).

In [31]:
import urllib.request, urllib.parse, urllib.error, json, re, datetime, sys, http.cookiejar
from got3 import models
from pyquery import PyQuery

In [42]:
username = 'pspca'
since = '2017-09-03'
until = '2017-09-07'

# max_tweets = 1

In [43]:
# this is the beginning of getTweets, including some of the default parameters
tweetCriteria = got.manager.TweetCriteria().setUsername(username).setSince(since).setUntil(until)

receiveBuffer = None  # parameter passed to getTweets
bufferLength=100 # parameter passed to getTweets
proxy=None # parameter passed to getTweets
refreshCursor = ''
results = []
resultsAux = []
cookieJar = http.cookiejar.CookieJar()

In [34]:
# this is all the functionality in getJsonResponse which is called inside getTweets
url = "https://twitter.com/i/search/timeline?f=tweets&q=%s&src=typd&%smax_position=%s"

urlGetData = ''
if hasattr(tweetCriteria, 'username'):
	urlGetData += ' from:' + tweetCriteria.username
	
if hasattr(tweetCriteria, 'since'):
	urlGetData += ' since:' + tweetCriteria.since
	
if hasattr(tweetCriteria, 'until'):
	urlGetData += ' until:' + tweetCriteria.until
	
if hasattr(tweetCriteria, 'querySearch'):
	urlGetData += ' ' + tweetCriteria.querySearch
	
if hasattr(tweetCriteria, 'lang'):
	urlLang = 'lang=' + tweetCriteria.lang + '&'
else:
	urlLang = ''
url = url % (urllib.parse.quote(urlGetData), urlLang, refreshCursor)
print(url)
headers = [
	('Host', "twitter.com"),
	('User-Agent', "Mozilla/5.0 (Windows NT 6.1; Win64; x64)"),
	('Accept', "application/json, text/javascript, */*; q=0.01"),
	('Accept-Language', "de,en-US;q=0.7,en;q=0.3"),
	('X-Requested-With', "XMLHttpRequest"),
	('Referer', url),
	('Connection', "keep-alive")
]

if proxy:
	opener = urllib.request.build_opener(urllib.request.ProxyHandler({'http': proxy, 'https': proxy}), urllib.request.HTTPCookieProcessor(cookieJar))
else:
	opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cookieJar))
opener.addheaders = headers

response = opener.open(url)
jsonResponse = response.read()

dataJson = json.loads(jsonResponse.decode())

https://twitter.com/i/search/timeline?f=tweets&q=%20from%3Apspca%20since%3A2017-09-06%20until%3A2017-09-07&src=typd&max_position=


In [ ]:
dataJson['items_html'].keys()

In [ ]:
# pprint(dataJson)

In [ ]:
# how to see if tweet is a reply
tweetPQ("div.tweet").attr("data-is-reply-to")

In [ ]:
# how to get username from tweet
tweetPQ("div.tweet").attr("data-screen-name")

### Now that I got json file, look at how getTweets gets the attributes

In [ ]:
refreshCursor = dataJson['min_position']
tweets = PyQuery(dataJson['items_html'])('div.js-stream-tweet')

for tweetHTML in tweets:
    tweetPQ = PyQuery(tweetHTML)
    tweet = models.Tweet()
    break

### Create data frame to see what gets outputted in a dataframe

In [44]:
tweet_list = got.manager.TweetManager.getTweets(tweetCriteria)

from collections import defaultdict
d = defaultdict(list)
attributes = ['date', 'favorites', 'retweets', 'hashtags', 'geo', 'id',
                  'mentions', 'text', 'urls', 'username', 'permalink', 'author_id']
for tweet in tweet_list:
    for att in attributes:
        d[att].append(eval("tweet." + att))

df = pd.DataFrame(d)
df

,author_id,date,favorites,geo,hashtags,id,mentions,permalink,retweets,text,urls,username
0,23483274,2017-09-06 14:15:24,10,,,905434261041373184,,https://twitter.com/PSPCA/status/9054342610413...,6,5 years is a long time to wait to go home. Pur...,http://ow.ly/JpvE30eW58l,PSPCA
1,23483274,2017-09-05 23:15:13,10,,#150for150,905207721447886848,,https://twitter.com/PSPCA/status/9052077214478...,4,"""I adopted Coco in January '13. I had been wat...",http://ow.ly/ZF5q30eW2Wk,PSPCA
2,23483274,2017-09-05 14:12:37,1,,#PetProject,905071170982219777,@CarolNewsbreak @CBSPhilly,https://twitter.com/PSPCA/status/9050711709822...,1,Have you wondered how your dog gets along w/ c...,http://ow.ly/UhfR30eVgje,PSPCA
3,23483274,2017-09-04 22:15:10,9,,#150for150,904830223241359361,,https://twitter.com/PSPCA/status/9048302232413...,1,"""Despite assurances he was going to live forev...",http://ow.ly/jN6G30eQTyK,PSPCA
4,23483274,2017-09-04 16:35:11,4,,,904744662191276039,@Petfinder_fnd @buildabear,https://twitter.com/PSPCA/status/9047446621912...,2,Thanks you to the @Petfinder_fnd on behalf of ...,,PSPCA
5,23483274,2017-09-04 14:05:20,12,,#Fishtown #Danville,904706951396941825,,https://twitter.com/PSPCA/status/9047069513969...,5,Happy Labor Day! Our Erie Avenue headquarters ...,,PSPCA
6,23483274,2017-09-03 22:20:07,7,,#150for150,904469080622686208,,https://twitter.com/PSPCA/status/9044690806226...,3,"""The PSPCA rescued Mazzy from an abandoned hou...",http://ow.ly/qxUp30eQRZ7,PSPCA
7,23483274,2017-09-03 14:15:14,32,,#pittie #furever,904347053928665088,,https://twitter.com/PSPCA/status/9043470539286...,26,"Special Pet Sunday! Meet Bruce, an adorable 3-...",http://ow.ly/hts530eQPEH,PSPCA
8,23483274,2017-09-03 11:35:05,5,,#petproject,904306751016755202,@CBSPhilly @CarolNewsbreak @JanCarabeoCBS3,https://twitter.com/PSPCA/status/9043067510167...,1,It's our favorite time of the week. Tune into ...,,PSPCA


In [37]:
# this adjusts the hour for time zone difference since API gives it in UTC
def adjust_hour(hour, offset):
    if hour >= offset:
        hour -= offset
    else:
        hour += 24 - offset
    return hour

In [38]:
# splits the datetime column into separate features
def add_date_features(df, offset=0):
    df['year'] = df.date.apply(lambda x: x.year)
    df['month'] = df.date.apply(lambda x: x.month)
    df['weekday'] = df.date.apply(lambda x: x.weekday())
    hours = df.date.apply(lambda x: x.hour)
    df['local_hour'] = hours.apply(adjust_hour, args=(offset,))
    
    return df

In [41]:
df = add_date_features(df, offset=4)
df

,author_id,date,favorites,geo,hashtags,id,mentions,permalink,retweets,text,urls,username,year,month,weekday,local_hour
0,23483274,2017-09-06 14:15:24,10,,,905434261041373184,,https://twitter.com/PSPCA/status/9054342610413...,6,5 years is a long time to wait to go home. Pur...,http://ow.ly/JpvE30eW58l,PSPCA,2017,9,2,10


In [45]:
df = pd.read_pickle('../data/all_tweets.p')

In [50]:
df.loc[(df['local_hour'] < 5) & (df['year'] == 2017)][['date', 'username', 'local_hour', 'year']]

,date,username,local_hour,year
7263,2017-04-09 04:15:06,PSPCA,0,2017
70599,2017-06-12 07:05:17,FMSPCA,1,2017
91182,2017-01-30 07:08:01,BC_SPCA,0,2017
91205,2017-01-26 07:01:04,BC_SPCA,0,2017
91231,2017-01-22 07:36:01,BC_SPCA,0,2017
91265,2017-01-16 07:37:01,BC_SPCA,0,2017
91311,2017-01-08 07:23:01,BC_SPCA,0,2017
91344,2017-01-02 07:01:01,BC_SPCA,0,2017
91349,2017-01-01 07:07:00,BC_SPCA,0,2017
91367,2017-02-26 07:23:01,BC_SPCA,0,2017


In [54]:
df.loc[df['year']==2008]

,author_id,date,favorites,hashtags,id,mentions,permalink,retweets,text,urls,username,year,month,weekday,local_hour,country
10739,16630213,2008-10-07 15:24:26,0,,949757786,,https://twitter.com/spcaoftexas/status/949757786,0,Big Dog Special! Only fifty dollars to adopt a...,,spcaoftexas,2008,10,1,10,usa
10740,16630213,2008-12-23 20:09:13,0,#jollyrancher,1074937042,,https://twitter.com/spcaoftexas/status/1074937042,0,Have you met Jolly Rancher? http://tinyurl.com...,,spcaoftexas,2008,12,1,15,usa
10741,16630213,2008-12-22 21:18:29,0,,1073029217,,https://twitter.com/spcaoftexas/status/1073029217,0,Looking for Last Minute Gifts? Check out www.s...,,spcaoftexas,2008,12,0,16,usa
10742,16630213,2008-12-21 20:43:24,0,#cupcake,1071042131,,https://twitter.com/spcaoftexas/status/1071042131,0,Have You Met Cupcake??? http://tinyurl.com/9s5...,,spcaoftexas,2008,12,6,15,usa
10743,16630213,2008-12-18 15:45:04,0,,1065108068,,https://twitter.com/spcaoftexas/status/1065108068,0,Help bring them Home for the Holidays: http://...,,spcaoftexas,2008,12,3,10,usa
10744,16630213,2008-12-17 15:18:03,0,,1062904509,,https://twitter.com/spcaoftexas/status/1062904509,0,"A Holiday Message from James Bias, SPCA of Tex...",,spcaoftexas,2008,12,2,10,usa
10745,16630213,2008-12-16 14:40:32,0,,1060683829,,https://twitter.com/spcaoftexas/status/1060683829,0,Keep your pets comfy and warm in this cold win...,,spcaoftexas,2008,12,1,9,usa
10746,16630213,2008-12-15 23:11:07,0,,1059503636,,https://twitter.com/spcaoftexas/status/1059503636,0,Marley and Me Photo Contest is Over. Visit htt...,,spcaoftexas,2008,12,0,18,usa
10747,16630213,2008-12-15 20:09:30,0,,1059177914,,https://twitter.com/spcaoftexas/status/1059177914,0,Don't Forget!! Today is the last day to submit...,,spcaoftexas,2008,12,0,15,usa
10748,16630213,2008-12-10 14:45:26,0,,1049225894,@730,https://twitter.com/spcaoftexas/status/1049225894,0,Get free advance screening passes to Marley & ...,,spcaoftexas,2008,12,2,9,usa


### Playing around with time zone converstions making datetime aware.

In [74]:
from datetime import datetime
import pytz

In [75]:
# get the time zones
utc_zone = pytz.timezone('UTC')
et_zone = pytz.timezone('US/Eastern')

In [69]:
# get a timestamp from the dataframe
test = df.date[0]
test

Timestamp('2009-03-30 17:19:45')

In [76]:
# converts pandas timestamp to datetime aware object
test_utc = test.to_pydatetime().replace(tzinfo=utc_zone)
test_utc

datetime.datetime(2009, 3, 30, 17, 19, 45, tzinfo=<UTC>)

In [77]:
# convert the utc to eastern
eastern = test_utc.astimezone(et_zone)
eastern

datetime.datetime(2009, 3, 30, 13, 19, 45, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)

In [104]:
# find a date that is outside dst
test2 = df.loc[(df['month'] == 1) & (df['username'] == 'PSPCA')].iloc[0,1]
test2

Timestamp('2010-01-26 15:31:43')

In [108]:
test2_utc = test2.to_pydatetime().replace(tzinfo=utc_zone)
test2_utc

datetime.datetime(2010, 1, 26, 15, 31, 43, tzinfo=<UTC>)

In [109]:
# IT WORKS!!!!!
eastern2 = test2_utc.astimezone(et_zone)
eastern2

datetime.datetime(2010, 1, 26, 10, 31, 43, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>)

In [112]:
# test the function to be used in the project

# get the time zones
et_zone = pytz.timezone('US/Eastern')
ct_zone = pytz.timezone('US/Central')
mt_zone = pytz.timezone('US/Mountain')
pt_zone = pytz.timezone('US/Pacific')

from datetime import datetime
import pytz
# function that replaces pandas timestamp with timezone adjusted datetime aware object
def convert_to_datetime_aware(date, tz=None):
    '''
    Parameters: 
        date - pandas UTC timestamp , 
        tz - a pytz time zone to convert to (default is None)
    
    Returns:
        datetime aware object in specified time zone (UTC if None)
    
    '''
    utc_zone = pytz.timezone('UTC')
    date = date.to_pydatetime().replace(tzinfo=utc_zone)
    
    if tz:
        return date.astimezone(tz)
    
    return date

In [126]:
date = convert_to_datetime_aware(test, et_zone)
date

datetime.datetime(2009, 3, 30, 13, 19, 45, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)

In [122]:
date.month

3

In [127]:
df_sfspca = pd.read_pickle('../data/sfspca_tweets.p')
df_sfspca.date.apply(convert_to_datetime_aware, args=(pt_zone,)).head(3)

0   2009-03-30 10:19:45-07:00
1   2009-03-28 14:49:03-07:00
2   2009-03-27 10:00:50-07:00
Name: date, dtype: datetime64[ns, US/Pacific]

In [128]:
df_sfspca.date.head(3)

0   2009-03-30 17:19:45
1   2009-03-28 21:49:03
2   2009-03-27 17:00:50
Name: date, dtype: datetime64[ns]

In [129]:
df_sfspca.date.apply(convert_to_datetime_aware).head(3)

0   2009-03-30 17:19:45+00:00
1   2009-03-28 21:49:03+00:00
2   2009-03-27 17:00:50+00:00
Name: date, dtype: datetime64[ns, UTC]

**Result:**  
The `pytz` looks to be working well, accounting for timezones and dst.

### New functionality to account for end date not being inclusive

In [ ]:
# ADJUSTMENT FOR UNTIL NOT BEING INCLUSIVE, SO START EACH MONTH WITH END OF PREVIOUS MONTH
#     years = ['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']
#     months = [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'),
#               ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]
#     total = 0
#     for year in years:
#         for month in months:
#             if year == '2008' and month[0] == '01':
#                 since = year + '-' + month[0] + '-01'
#             else:
#                 since = until
#             until = year + '-' + month[0] + '-' + month[1]
#             print(since, until)

### 9/6/17 - Playing around with Twittercounter API

In [1]:
import pandas as pd
from urllib.parse import urlencode
import requests

In [2]:
df_sfspca = pd.read_pickle('../data/sfspca_tweets.p')
df_pspca = pd.read_pickle('../data/pspca_tweets.p')
df_houston = pd.read_pickle('../data/houston_tweets.p')
df_texas = pd.read_pickle('../data/texas_tweets.p')
df_tulsa = pd.read_pickle('../data/tulsa_tweets.p')
df_richmond = pd.read_pickle('../data/richmond_tweets.p')
df_ontario = pd.read_pickle('../data/ontario_tweets.p')
df_alberta = pd.read_pickle('../data/alberta_tweets.p')
df_bc = pd.read_pickle('../data/bc_tweets.p')

In [16]:
url = 'http://api.twittercounter.com'
api_key = '7e7c4964440d3b1f61cdc34a0fbb3576'
user_id = '23483274'
r = requests.get(url + '?' + urlencode({'apikey':api_key, 'twitter_id':user_id}))

In [17]:
data = r.json()
data.keys()

dict_keys(['version', 'twitter_id', 'username', 'url', 'avatar', 'name', 'description', 'timezone', 'location', 'last_tweet_id', 'followers_current', 'date_updated', 'follow_days', 'started_followers', 'growth_since', 'average_growth', 'tomorrow', 'next_month', 'followers_yesterday', 'rank', 'followersperdate', 'last_update'])

In [18]:
data['followersperdate']

{'date2016-07-12': 6339,
 'date2016-07-16': 6347,
 'date2016-07-18': 6356,
 'date2016-08-18': 6520,
 'date2016-09-04': 6651,
 'date2016-09-16': 6724,
 'date2016-11-02': 6875,
 'date2016-11-03': 6885,
 'date2016-11-11': 6914,
 'date2016-12-09': 7001,
 'date2017-02-17': 7249,
 'date2017-04-19': 7475,
 'date2017-04-20': 7475,
 'date2017-06-18': 7650}

In [19]:
data['follow_days']

'2885'

**Result:**  
Can't go back far enough so stretch goal would be to subset the data so that it only goes back far enough to track number of followers. Then I can include that in my metric.